In [25]:
'''
Author : Thomas Lecaé
Date :2023/04
Summary : this script takes infos from your database to connect and then analyses tables 
contained in a specific schema and fills a LateX file with their 
metadata (table_name, column_name, data_type, character_maximum_length, is_nullable).
Input : PostgreSQL database connection details
Returns : Latex file
'''
import psycopg2
from tabulate import tabulate
import os

# Manage dirs 
parent_dir = os.path.abspath(os.path.join(os.getcwd())) 
print(parent_dir)
output_dir = os.path.join(parent_dir, "output")
print(output_dir)
# Create the output dir if it doesn't exist
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
# PostgreSQL database connection details
host = "localhost"
database = "postgres"
user = "postgres"
password = "admin"
port = "5432"

# Name of the schema to analyze
schema_name = "cvi"

# Connect to PostgreSQL database
conn = psycopg2.connect(
    host=host,
    database=database,
    user=user,
    password=password,
    port=port
)

# Create a cursor
cur = conn.cursor()

# Retrieve metadata for each table in the schema
cur.execute("""
    SELECT table_name, column_name, data_type, character_maximum_length, is_nullable
    FROM information_schema.columns
    WHERE table_schema = %s
    ORDER BY table_name, ordinal_position
""", (schema_name,))

# Group the metadata by table
table_metadata = {}
for row in cur.fetchall():
    table_name, column_name, data_type, character_maximum_length, is_nullable = row
    if table_name not in table_metadata:
        table_metadata[table_name] = []
    table_metadata[table_name].append((column_name, data_type, character_maximum_length, is_nullable))

# Generate LaTeX file with table metadata
with open(os.path.join(output_dir, "table_metadata.tex"), "w") as f:
    for table_name, metadata in table_metadata.items():
        headers = ["Column Name", "Data Type", "Max Length", "Nullable"]
        table = tabulate(metadata, headers=headers, tablefmt="latex_raw")
        f.write("\\subsection*{Table: " + table_name + "}\n")
        f.write(table + "\n")
        


c:\_DEV\test\metadata
c:\_DEV\test\metadata\output
